# Process of Scrapping all Video data from a channel

In [ ]:
"""
1) Create API key from below link and enable it in your project.
   https://console.cloud.google.com/
1) run below command on anaconda prompt 
   pip install --upgrade google-api-python-client
2) Find the channel Id of youtube channel and paste it into below link to understand code provided by Youtube API to get required data.
   https://developers.google.com/youtube/v3/docs/channels/list
For example I am getting data of one youtube channel for which Id is="UCeVMnSShP_Iviwkknt83cww"  #Code with harry uploadId=UUeVMnSShP_Iviwkknt83cww
3) To Scrap all video data from above channel Find upload ID first.
4) From upload Id extract information of all video IDs.
5) Use all video IDs to get the data of all videos posted by this particular channel
"""

## Step-1 : Connecting from youtube API.

In [19]:
import googleapiclient.discovery

In [20]:
api_service_name ="youtube"
api_version ="v3"
DEVELOPER_KEY ="APIKey"

In [21]:
youtube = googleapiclient.discovery.build(
api_service_name, api_version, developerKey = DEVELOPER_KEY)

In [22]:
request = youtube.channels().list(
        part="snippet,statistics,contentDetails",
        id="UCeVMnSShP_Iviwkknt83cww"
    )
response = request.execute()
print(response)

{'kind': 'youtube#channelListResponse', 'etag': 'D_iCP0k3XN57ssJd190_DqGGe3M', 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5}, 'items': [{'kind': 'youtube#channel', 'etag': '1veYNq1YE6fLXSOyON7vD-27ODA', 'id': 'UCeVMnSShP_Iviwkknt83cww', 'snippet': {'title': 'CodeWithHarry', 'description': 'Code With Harry is my attempt to teach basics and those coding techniques to people in short time which took me ages to learn.\nAt Code With Harry, I provide a quick and to the point demo along with resources of anything and everything I teach. Source code and other resources are hosted on my website CodeWithHarry.com. I provide source code(if any) in the description of every video.\nQuality programming videos in Hindi :)\nBusiness enquiries: harry@codewithharry.com\n', 'customUrl': '@codewithharry', 'publishedAt': '2018-04-28T14:50:54Z', 'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/ytc/AIf8zZTNiWLD2-Qw1eWFmRJkGBzfI-8oFNmYk5tBnpNurQ=s88-c-k-c0x00ffffff-no-rj', 'width': 88, 'height'

## Step-2: Finding the upload if for a particular channel¶

In [23]:
response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

'UUeVMnSShP_Iviwkknt83cww'

## Step-3: Get the Video_id from playlist_id(upload_id) code

In [ ]:
request = youtube.playlistItems().list(
        part="contentDetails",
        playlistId='UUFgPJgsmH8u67SvvLK-tLGg',#"UUeVMnSShP_Iviwkknt83cww",
        maxResults=50
)
response = request.execute()
print(response)

## Step-4: With Step3 we can get max of 50 video ids at once so we need to modify code to get if a channel contains more than 50 Videos, we will use nextPageToken Attribute available in Youtube API

In [26]:
def get_videoId(Id):
    request = youtube.playlistItems().list(
            part="snippet,contentDetails",
            playlistId=Id,
            maxResults=50
    )
    response= request.execute()
    Video_id=[]
    for i in range(len(response['items'])):
        Video_id.append(response['items'][i]["contentDetails"]["videoId"])
    next_page=True
    while next_page:
        if response.get('nextPageToken') is None:
            next_page=False
        else:
            request = youtube.playlistItems().list(
            part="snippet,contentDetails",
            playlistId=Id,
            pageToken=response.get('nextPageToken'),   
            maxResults=50)
            response= request.execute()
            for i in range(len(response['items'])):
                Video_id.append(response['items'][i]["contentDetails"]["videoId"])
            pageToken=response.get('nextPageToken')    
    return Video_id
    

In [27]:
Total_videosID=get_videoId("UUeVMnSShP_Iviwkknt83cww")

## Step-5: With Video_ids we can get the details of each videos.## Step-5: With Video_ids we can get the details of each videos.

In [30]:
def get_data(VideoIds):
    finaldata=[]
    for i in range(0,len(VideoIds),50):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=','.join(VideoIds[i:i+50])
        )
        response = request.execute()
        for i in response['items']:
            data=dict(Title=i['snippet']['title'],
                      Published_date=i['snippet']['publishedAt'],
                      Total_views=i['statistics']["viewCount"],
                      Total_likes=i['statistics']["likeCount"],
                      Totalcomment=i['statistics']["commentCount"])
            finaldata.append(data)
    return finaldata

In [32]:
finalop=get_data(Total_videosID)

In [33]:
import pandas as pd
Dataframe=pd.DataFrame(finalop)
Dataframe

,Title,Published_date,Total_views,Total_likes,Totalcomment
0,Which Code Editor to Use?,2024-02-09T16:30:05Z,13729,1602,52
1,Exercise 18 - Solution & Shoutouts | Sigma Web...,2024-02-08T16:14:04Z,9013,431,124
2,Handling Events in React | Sigma Web Developme...,2024-02-07T17:05:50Z,10976,646,156
3,👨‍💻 Should Kids learn Coding?,2024-02-07T07:14:21Z,35824,3272,65
4,Exercise 18 - Display the Cards | Sigma Web De...,2024-02-06T16:09:04Z,12271,670,165
...,...,...,...,...,...
2299,Learn Python In Hindi In One Video - हिंदी में,2018-06-04T13:59:02Z,338777,10566,622
2300,[Solved] The term 'pip' is not recognized as t...,2018-06-01T07:02:50Z,397623,7715,844
2301,JavaScript Tutorial,2018-05-13T16:22:37Z,413891,10530,575
2302,[Hindi] Which Programming Language to Learn Fi...,2018-05-12T09:55:59Z,51417,1708,125


## Changing the Datatype as all extracted data will be object

In [34]:
Dataframe['Published_date']=pd.to_datetime(Dataframe['Published_date'])
Dataframe['Total_views']=pd.to_numeric(Dataframe['Total_views'])
Dataframe['Total_likes']=pd.to_numeric(Dataframe['Total_likes'])
Dataframe['Totalcomment']=pd.to_numeric(Dataframe['Totalcomment'])
Dataframe

,Title,Published_date,Total_views,Total_likes,Totalcomment
0,Which Code Editor to Use?,2024-02-09 16:30:05+00:00,13729,1602,52
1,Exercise 18 - Solution & Shoutouts | Sigma Web...,2024-02-08 16:14:04+00:00,9013,431,124
2,Handling Events in React | Sigma Web Developme...,2024-02-07 17:05:50+00:00,10976,646,156
3,👨‍💻 Should Kids learn Coding?,2024-02-07 07:14:21+00:00,35824,3272,65
4,Exercise 18 - Display the Cards | Sigma Web De...,2024-02-06 16:09:04+00:00,12271,670,165
...,...,...,...,...,...
2299,Learn Python In Hindi In One Video - हिंदी में,2018-06-04 13:59:02+00:00,338777,10566,622
2300,[Solved] The term 'pip' is not recognized as t...,2018-06-01 07:02:50+00:00,397623,7715,844
2301,JavaScript Tutorial,2018-05-13 16:22:37+00:00,413891,10530,575
2302,[Hindi] Which Programming Language to Learn Fi...,2018-05-12 09:55:59+00:00,51417,1708,125
